Если что, работаю не в коллабе, а на своем ПК. Обучаю на ГПУ

In [ ]:
import matplotlib.pyplot as plt
from keras.optimizers import SGD
from tensorflow import keras
from keras import layers, regularizers
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
import PIL.Image

In [ ]:
dataset_path = 'DatasetNew/'
img_size = 96
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1 / 255.0,
    rotation_range=30,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.1,
    # zoom_range=0.1,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1 / 255.0)

test_datagen = ImageDataGenerator(rescale=1 / 255.0)

train_generator = train_datagen.flow_from_directory(
    directory=dataset_path + 'train/',
    target_size=(img_size, img_size),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)
valid_generator = val_datagen.flow_from_directory(
    directory=dataset_path + 'valid/',
    target_size=(img_size, img_size),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)
test_generator = test_datagen.flow_from_directory(
    directory=dataset_path + 'test/',
    target_size=(img_size, img_size),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

In [ ]:
def prepare_model():
    act = 'relu'
    opt = SGD(learning_rate=0.01, nesterov=True)

    model = keras.Sequential([
        Conv2D(64, (3, 3), activation=act, input_shape=(img_size, img_size, 3)),
        MaxPooling2D((2, 2), strides=2),
        BatchNormalization(),

        Conv2D(128, (3, 3), activation=act),
        MaxPooling2D((2, 2), strides=2),
        BatchNormalization(),

        Conv2D(256, (3, 3), activation=act),
        Conv2D(256, (3, 3), activation=act),
        MaxPooling2D((2, 2), strides=2),
        BatchNormalization(),

        Flatten(),
        Dense(32, activation=act, kernel_regularizer=regularizers.l2(0.01)),
        Dropout(0.25),
        Dense(32, activation=act, kernel_regularizer=regularizers.l2(0.01)),
        Dropout(0.4),
        Dense(8, activation='softmax')
    ])
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=['accuracy'])
    return model


epochs = 30
model = prepare_model()
model.summary()
his = model.fit(train_generator, validation_data=valid_generator, epochs=epochs)

In [ ]:
acc = his.history['accuracy']
val_acc = his.history['val_accuracy']
loss = his.history['loss']
val_loss = his.history['val_loss']
epochs_range = range(epochs)
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()